# merging and cleaning processed paediatric Visium data - post cell2location deconvolution

In [ ]:
import numpy as np
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sb
import re
import os
import scipy.stats
from numpy import asarray as ar
from collections import Counter
import scvi
import anndata as ad
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # to show output from all the lines in a cells
pd.set_option('display.max_column',None) # display all the columns in pandas
pd.options.display.max_rows = 100

from datetime import date
today = str(date.today())
sc.settings.verbosity = 1
sc.logging.print_version_and_date()
%load_ext autoreload
%autoreload 2

In [ ]:
## For correct plotting of the images
import matplotlib
from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42
sc.settings.set_figure_params(dpi = 150, color_map = 'RdPu', dpi_save = 150, vector_friendly = True, format = 'pdf')

In [ ]:
# output folder 
import os
os.getcwd()

# Orgenize analysis object 
this is divided to 4 steps:
1) load cell2loaction object "sp" which has only HVG genes 
2) load merged raw visium object "adata_vis" 
3) create a new object with all annotations and all raw genes
4) update tissue tag annotations from to generate a final annotated object 

In [ ]:
# load c2l object 
adata_paed = sc.read('')
# adata_paed

In [ ]:
# add 5% quantile, representing confident cell abundance, 'at least this amount is present', 
# to adata.obs with nice names for plotting
adata_paed.obs[adata_paed.uns['mod']['factor_names']] = adata_paed.obsm['q05_cell_abundance_w_sf']
adata_paed.obs['tot_cell_abundance'] = adata_paed.uns["mod"]["post_sample_means"]["w_sf"].sum(1).flatten()
adata_paed.obs['detection_sensit']  = adata_paed.uns["mod"]["post_sample_q05"]["detection_y_s"]
# adata_paed

In [ ]:
# recover original raw object and only recover model and annotations
adata_vis_all = sc.read("/nfs/team205/ny1/ThymusSpatialAtlas/Figure_2_nat/Thymus_atlas_v2_Visium_raw_2023-08-02.h5ad")
adata_vis = adata_vis_all[adata_vis_all.obs['Age_group'] != 'fetal'].copy()
del adata_vis_all
# adata_vis

In [ ]:
adata_paed_full = ad.AnnData(X = adata_vis.X, obs=adata_paed.obs, var=adata_vis.var, uns=adata_paed.uns, obsm=adata_paed.obsm)
del adata_vis
del adata_paed
# adata_paed_full

# embed new annotations 

In [ ]:
# remove outdated annotations 
annotations_to_remove = ['L2_dist_Annotation_lv_0_Artifacts', 'L2_dist_Annotation_lv_0_Background', 
                         'L2_dist_Annotation_lv_0_Cortex', 'L2_dist_Annotation_lv_0_Edge', 'L2_dist_Annotation_lv_0_Medulla', 
                         'L2_dist_log10_Annotation_lv_1_HS', 'Annotation_lv_1', 'L2_dist_log10_Annotation_lv_1_Lymph', 
                         'L2_dist_log10_Annotation_lv_1_PVS', 'L2_dist_log10_Annotation_lv_1_Unassigned', 'Annotation_lobules', 
                         'L2_dist_Annotation_lv_0_Unassigned', 'Annotation_lobules_0', 
                         'Annotation_lobules_1', 'L2_dist_log10_Annotation_lv_1_Fat', 'Leiden_0_5', 
                         'L2_dist_log10_Annotation_lv_1_Large vessel']

adata_paed_full.obs.drop(columns=annotations_to_remove, errors='ignore', inplace=True)
adata_paed_full.obs.rename(columns={'Cortico_Medullar_Axis': 'cma_v1'}, inplace=True)
adata_paed_full.obs.rename(columns={'Annotation_lv_0': 'old_annotations_level_0'}, inplace=True)


# adata_paed_full

In [ ]:
# assign nice colors 

In [ ]:
# specify the colors for your categories
# color mapping
colors_dict = {'unassigned': 'gray', 'PVS': 'red', 'HS': 'green', 'vessels': 'blue', 'lymph':'orange','Cortex': 'cyan', 'Medulla': 'green', 'Edge': 'brown', 'Artifacts': 'black'}



# get the categories in your data
categories = adata_paed_full.obs['annotations_level_0'].cat.categories

# map the colors to the categories
colors = []
for cat in categories:
    try:
        colors.append(colors_dict[cat])
    except KeyError:  # cat is not found in colors_dict
        colors.append('gray')  # use a default color

# assign the colors
adata_paed_full.uns['annotations_level_0_colors'] = colors

# get the categories in your data
categories = adata_paed_full.obs['annotations_level_1'].cat.categories

# map the colors to the categories
colors = []
for cat in categories:
    try:
        colors.append(colors_dict[cat])
    except KeyError:  # cat is not found in colors_dict
        colors.append('gray')  # use a default color

# assign the colors
adata_paed_full.uns['annotations_level_1_colors'] = colors



In [ ]:
# adata_paed_full

In [ ]:
adata_paed_full.write_h5ad('')